# Transformer Encoder-Only - UMAP Embeddings (Sistema G4)

**Objetivo:** Clasificación de secuencias temporales con Transformer encoder-only usando embeddings UMAP

**Dataset:** 868 videos, 96 frames, 128 features/frame (UMAP embeddings)  
**Clases:** 30 (ASL - American Sign Language)  
**Hardware:** RTX 5050 Laptop (8GB) / GTX 1660 Super (6GB)

---

## 🎯 Sistema de Gestión de Experimentos G4

Este notebook genera automáticamente **32 archivos** organizados en:
- `G4-RESULTS/` - Baseline (dropout 0.1)
- `G4-RESULTS-CLASS-WEIGHTS/` - Class Weights + Dropout 0.3
- `G4-RESULTS-LABEL-SMOOTH/` - Label Smoothing 0.1 + Dropout 0.3
- 2 archivos de comparación en ROOT_PATH

Cada experimento genera 10 archivos: best_model.pt, config.json, training_log.csv, metrics.csv, per_class_metrics.csv, confusion_matrix.csv, confusion_matrix.png, training_curves.png, per_class_analysis.png, RESUMEN.txt

## 📁 Sistema de Detección Automática de Rutas (G4)

El notebook detecta automáticamente en qué carpeta se encuentra y configura las rutas:

- Si detecta `UMAP` en la ruta → Usa carpeta `G4-EMBEDDING FRAME A FRAME UMAP/`
- Si detecta `GCN` o `Embeddings` → Usa carpeta `G4-EMBEDDING FRAME A FRAME GCN/`
- Si detecta `JSON` o `NORM` → Usa carpeta `G4-JSON-NORM/`

Los archivos se guardan dentro de subcarpetas según el experimento:
- `G4-RESULTS` (baseline)
- `G4-RESULTS-CLASS-WEIGHTS` (experimento 1)
- `G4-RESULTS-LABEL-SMOOTH` (experimento 2)

In [55]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, top_k_accuracy_score
)
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Configuración dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# 🔧 CONFIGURACIÓN AUTOMÁTICA DE RUTAS Y EXPERIMENTOS (G4)
BASE_PATHS = {
    'umap': Path(r'C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP'),
    'gcn': Path(r'C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME GCN'),
    'json': Path(r'C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-JSON-NORM')
}

# Detectar automáticamente el tipo de experimento
current_notebook = Path.cwd()
notebook_name_lower = str(current_notebook).lower()

if 'umap' in notebook_name_lower:
    DETECTED_MODE = 'umap'
elif 'embedding' in notebook_name_lower or 'gcn' in notebook_name_lower:
    DETECTED_MODE = 'gcn'
elif 'json' in notebook_name_lower or 'norm' in notebook_name_lower:
    DETECTED_MODE = 'json'
else:
    DETECTED_MODE = 'umap'  # Default para este notebook

ROOT_PATH = BASE_PATHS[DETECTED_MODE]
AUTO_DETECTED = True

print(f"\n{'='*80}")
print(f"🎯 SISTEMA DE DETECCIÓN AUTOMÁTICA DE RUTAS G4")
print(f"{'='*80}")
print(f"📂 Notebook detectado: {current_notebook.name}")
print(f"🔍 Modo detectado: {DETECTED_MODE.upper()}")
print(f"📁 ROOT_PATH: {ROOT_PATH}")
print(f"✅ Detección automática: {'ACTIVADA' if AUTO_DETECTED else 'DESACTIVADA'}")
print(f"{'='*80}\n")

# Configuraciones de experimentos (G4)
EXPERIMENT_CONFIGS = {
    'baseline': {
        'folder_name': 'G4-RESULTS',
        'description': 'Baseline - Dropout 0.1',
        'dropout': 0.1,
        'use_class_weights': False,
        'label_smoothing': 0.0
    },
    'class_weights': {
        'folder_name': 'G4-RESULTS-CLASS-WEIGHTS',
        'description': 'Class Weights + Dropout 0.3',
        'dropout': 0.3,
        'use_class_weights': True,
        'label_smoothing': 0.0
    },
    'label_smoothing': {
        'folder_name': 'G4-RESULTS-LABEL-SMOOTH',
        'description': 'Label Smoothing 0.1 + Dropout 0.3',
        'dropout': 0.3,
        'use_class_weights': False,
        'label_smoothing': 0.1
    }
}

# Seleccionar experimento (MODIFICAR AQUÍ PARA CAMBIAR EXPERIMENTO)
EXPERIMENT_TYPE = 'baseline'  # Opciones: 'baseline', 'class_weights', 'label_smoothing'

current_config = EXPERIMENT_CONFIGS[EXPERIMENT_TYPE]
output_dir = ROOT_PATH / current_config['folder_name']
output_dir.mkdir(parents=True, exist_ok=True)

print(f"⚙️  CONFIGURACIÓN ACTUAL:")
print(f"  • Tipo de experimento: {EXPERIMENT_TYPE}")
print(f"  • Descripción: {current_config['description']}")
print(f"  • Directorio de salida: {output_dir}")
print(f"  • Dropout: {current_config['dropout']}")
print(f"  • Class Weights: {current_config['use_class_weights']}")
print(f"  • Label Smoothing: {current_config['label_smoothing']}")
print(f"{'='*80}\n")

Device: cuda
GPU: NVIDIA GeForce RTX 5050 Laptop GPU
VRAM: 8.55 GB

🎯 SISTEMA DE DETECCIÓN AUTOMÁTICA DE RUTAS G4
📂 Notebook detectado: transformer-asl-classification
🔍 Modo detectado: UMAP
📁 ROOT_PATH: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP
✅ Detección automática: ACTIVADA

⚙️  CONFIGURACIÓN ACTUAL:
  • Tipo de experimento: baseline
  • Descripción: Baseline - Dropout 0.1
  • Directorio de salida: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS
  • Dropout: 0.1
  • Class Weights: False
  • Label Smoothing: 0.0



In [56]:
# 🧹 LIMPIEZA DE CARPETAS PREVIAS
# Esta celda elimina carpetas de experimentos previos antes de ejecutar los 3 experimentos

import shutil

print(f"\n{'='*80}")
print(f"🧹 LIMPIEZA DE CARPETAS PREVIAS")
print(f"{'='*80}")

# Carpetas de resultados G4 a limpiar
results_folders = [
    'G4-RESULTS',
    'G4-RESULTS-CLASS-WEIGHTS',
    'G4-RESULTS-LABEL-SMOOTH'
]

# Limpiar dentro de ROOT_PATH
cleaned_count = 0
for folder in results_folders:
    folder_path = ROOT_PATH / folder
    if folder_path.exists():
        shutil.rmtree(folder_path)
        print(f"  ✓ Eliminado: {folder_path}")
        cleaned_count += 1

# También limpiar carpetas antiguas en el directorio raíz del proyecto
project_root = Path.cwd()
old_folders = [
    'g5.0_umap',
    'g6_class_weights',
    'g7_label_smooth',
    'umap_baseline',
    'results_umap',
    'output_videos',
    'temp_results',
    'old_results',
    'results'
]

for folder in old_folders:
    folder_path = project_root / folder
    if folder_path.exists():
        shutil.rmtree(folder_path)
        print(f"  ✓ Eliminado (raíz): {folder_path}")
        cleaned_count += 1

if cleaned_count == 0:
    print("  ℹ️  No hay carpetas previas para limpiar")

print(f"✅ Limpieza completada ({cleaned_count} carpetas eliminadas)")

# Recrear el directorio de salida actual después de la limpieza
output_dir.mkdir(parents=True, exist_ok=True)
print(f"📁 Directorio de salida recreado: {output_dir}\n")


🧹 LIMPIEZA DE CARPETAS PREVIAS
  ✓ Eliminado: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS
  ✓ Eliminado (raíz): c:\Users\isaiy\Documents\modelo prueba embedings\transformer-asl-classification\g5.0_umap
✅ Limpieza completada (2 carpetas eliminadas)
📁 Directorio de salida recreado: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS



In [57]:
# 1. CARGAR DATASET UMAP
dataset_path = Path('./daataset/G2_GCN_embeddings_separado_128d.npz')
data = np.load(dataset_path, allow_pickle=True)

# Inspeccionar las claves disponibles
print("Claves disponibles en el archivo .npz:")
print(data.files)

# Mostrar también el tipo y forma de cada array
for key in data.files:
    print(f"\n{key}:")
    print(f"  Tipo: {type(data[key])}")
    if hasattr(data[key], 'shape'):
        print(f"  Forma: {data[key].shape}")

# Una vez que veas las claves correctas, reemplaza 'X' con el nombre correcto
# X = data['nombre_correcto']

Claves disponibles en el archivo .npz:
['X', 'y', 'filenames', 'class_names', 'masks']

X:
  Tipo: <class 'numpy.ndarray'>
  Forma: (864, 96, 128)

y:
  Tipo: <class 'numpy.ndarray'>
  Forma: (864,)

filenames:
  Tipo: <class 'numpy.ndarray'>
  Forma: (864,)

class_names:
  Tipo: <class 'numpy.ndarray'>
  Forma: (30,)

masks:
  Tipo: <class 'numpy.ndarray'>
  Forma: (864, 96)


In [58]:
# ✅ VALIDACIÓN - CARGAR ARCHIVO GCN CON ESTRUCTURA CORRECTA
print(f"\n{'='*80}")
print(f"✅ VALIDACIÓN - ESTRUCTURA DE G2_GCN_embeddings_separado_128d.npz")
print(f"{'='*80}\n")

dataset_path_gcn = Path('./daataset/G2_GCN_embeddings_separado_128d.npz')
data_gcn = np.load(dataset_path_gcn, allow_pickle=True)

# 📋 Claves disponibles
print("📋 CLAVES DISPONIBLES EN EL ARCHIVO:")
print(f"   {list(data_gcn.files)}\n")

# 📊 INFORMACIÓN DETALLADA DE CADA ARRAY
print("📊 INFORMACIÓN DETALLADA DE CADA ARRAY:\n")

expected_keys = {
    'X': (864, 96, 128),
    'y': (864,),
    'filenames': (864,),
    'class_names': (30,),
    'masks': (864, 96)
}

validation_results = {}

for key in expected_keys.keys():
    if key in data_gcn.files:
        array = data_gcn[key]
        actual_shape = array.shape
        expected_shape = expected_keys[key]
        
        is_valid = actual_shape == expected_shape
        status = "✓ VÁLIDO" if is_valid else "✗ ERROR"
        
        validation_results[key] = {
            'exists': True,
            'valid': is_valid,
            'dtype': array.dtype,
            'shape': actual_shape,
            'expected_shape': expected_shape
        }
        
        print(f"{status} - {key}:")
        print(f"   Tipo: {type(array).__name__}")
        print(f"   dtype: {array.dtype}")
        print(f"   Forma actual: {actual_shape}")
        print(f"   Forma esperada: {expected_shape}")
        
        if key == 'X':
            print(f"   Rango de valores: [{array.min():.4f}, {array.max():.4f}]")
        elif key == 'y':
            print(f"   Valores únicos: {np.unique(array).tolist()}")
            print(f"   Distribución de clases: {dict(zip(*np.unique(array, return_counts=True)))}")
        elif key == 'class_names':
            print(f"   Nombres de clases: {array.tolist()}")
        elif key == 'filenames':
            print(f"   Primeros 3 nombres: {array[:3].tolist()}")
        elif key == 'masks':
            unique_vals = np.unique(array)
            print(f"   Valores únicos en masks: {unique_vals}")
            print(f"   Cantidad de 1.0: {np.sum(array == 1.0)}, Cantidad de 0.0: {np.sum(array == 0.0)}")
        
        print()
    else:
        validation_results[key] = {'exists': False, 'valid': False}
        print(f"✗ FALTANTE - {key}:")
        print(f"   La clave no existe en el archivo\n")

# 🎯 RESUMEN DE VALIDACIÓN
print(f"\n{'='*80}")
print("🎯 RESUMEN DE VALIDACIÓN:")
print(f"{'='*80}\n")

all_valid = all(v.get('valid', False) for v in validation_results.values())
total_keys = len(expected_keys)
valid_keys = sum(1 for v in validation_results.values() if v.get('valid', False))

print(f"Claves válidas: {valid_keys}/{total_keys}")
for key, result in validation_results.items():
    status = "✓" if result.get('valid', False) else "✗"
    print(f"  {status} {key}: {result.get('shape', 'FALTANTE')}")

if all_valid:
    print(f"\n✅ ESTRUCTURA VÁLIDA - El archivo contiene todas las claves con las formas correctas")
else:
    print(f"\n❌ ESTRUCTURA INVÁLIDA - Revisar los errores arriba")

# 📦 CARGAR LOS DATOS
print(f"\n{'='*80}")
print("📦 CARGANDO DATOS:")
print(f"{'='*80}\n")

X = data_gcn['X']
y = data_gcn['y']
filenames = data_gcn['filenames']
class_names = data_gcn['class_names']
masks = data_gcn['masks']

print(f"✓ X (embeddings): {X.shape} | dtype: {X.dtype}")
print(f"✓ y (etiquetas): {y.shape} | unique classes: {len(np.unique(y))}")
print(f"✓ filenames: {filenames.shape} | samples: {len(filenames)}")
print(f"✓ class_names: {class_names.shape} | total clases: {len(class_names)}")
print(f"✓ masks: {masks.shape} | dtype: {masks.dtype}")

print(f"\n✅ CARGA COMPLETADA EXITOSAMENTE\n")


✅ VALIDACIÓN - ESTRUCTURA DE G2_GCN_embeddings_separado_128d.npz

📋 CLAVES DISPONIBLES EN EL ARCHIVO:
   ['X', 'y', 'filenames', 'class_names', 'masks']

📊 INFORMACIÓN DETALLADA DE CADA ARRAY:

✓ VÁLIDO - X:
   Tipo: ndarray
   dtype: float32
   Forma actual: (864, 96, 128)
   Forma esperada: (864, 96, 128)
   Rango de valores: [-30.6874, 26.5063]

✓ VÁLIDO - y:
   Tipo: ndarray
   dtype: int64
   Forma actual: (864,)
   Forma esperada: (864,)
   Valores únicos: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
   Distribución de clases: {0: 76, 1: 10, 2: 22, 3: 22, 4: 18, 5: 18, 6: 28, 7: 18, 8: 22, 9: 16, 10: 12, 11: 18, 12: 28, 13: 24, 14: 28, 15: 18, 16: 18, 17: 82, 18: 76, 19: 18, 20: 28, 21: 18, 22: 18, 23: 22, 24: 18, 25: 68, 26: 22, 27: 64, 28: 18, 29: 16}

✓ VÁLIDO - filenames:
   Tipo: ndarray
   dtype: <U42
   Forma actual: (864,)
   Forma esperada: (864,)
   Primeros 3 nombres: ['Adiós\\Adiós_1.json', 'Adiós\\Adi

In [59]:
# 1. CARGAR DATASET UMAP
dataset_path = Path('./daataset/G2_umap_segments_separado.npz')
data = np.load(dataset_path, allow_pickle=True)

# Cargar los arrays con las claves correctas
hands_segments = data['hands_segments']  # (864, 12, 100)
pose_segments = data['pose_segments']    # (864, 12, 100)
face_segments = data['face_segments']    # (864, 12, 100)
masks = data['masks']                     # (864, 96)
metadata = data['metadata']               # (864,)

print(f"Hands segments: {hands_segments.shape}")
print(f"Pose segments: {pose_segments.shape}")
print(f"Face segments: {face_segments.shape}")
print(f"Masks: {masks.shape}")
print(f"Metadata: {metadata.shape}")

# Concatenar los 3 segmentos (hands, pose, face) en el eje de características
# De (864, 12, 100) cada uno -> (864, 12, 300)
X = np.concatenate([hands_segments, pose_segments, face_segments], axis=2)

# Expandir para tener 96 segmentos (repitiendo cada segmento 8 veces: 12 * 8 = 96)
X_expanded = np.repeat(X, 8, axis=1)  # (864, 96, 300)

# Extraer las categorías del metadata y crear mapeo a números
categorias = [item['categoria'] for item in metadata]
categorias_unicas = sorted(list(set(categorias)))
categoria_a_idx = {cat: idx for idx, cat in enumerate(categorias_unicas)}

# Convertir categorías a índices numéricos
y = np.array([categoria_a_idx[cat] for cat in categorias])

# Guardar nombres de archivos
filenames = np.array([item['file'] for item in metadata])

print(f"\nDataset shape: X={X_expanded.shape}, y={y.shape}")
print(f"Masks shape: {masks.shape}")
print(f"Filenames: {len(filenames)}")
print(f"Classes: {len(categorias_unicas)}")
print(f"Categorías: {categorias_unicas}")

# Información de las clases
unique_classes, class_counts = np.unique(y, return_counts=True)
print(f"\nDistribución de clases:")
for cls, count in zip(unique_classes, class_counts):
    print(f"  Clase {cls} ({categorias_unicas[cls]}): {count} muestras")

# Renombrar para compatibilidad con el resto del notebook
X = X_expanded

Hands segments: (864, 12, 100)
Pose segments: (864, 12, 100)
Face segments: (864, 12, 100)
Masks: (864, 96)
Metadata: (864,)

Dataset shape: X=(864, 96, 300), y=(864,)
Masks shape: (864, 96)
Filenames: 864
Classes: 30
Categorías: ['Adiós', 'Buenas noches', 'Buenas tardes', 'Buenos días', 'Clase', 'Comenzar', 'Compañero', 'Cuaderno', 'Cómo está', 'Deberes', 'Disculpa', 'Entender', 'Escribir', 'Escuchar', 'Estudiante', 'Examen', 'Explicar', 'Gracias', 'Hola', 'Lección', 'Leer', 'Libro', 'Lápiz', 'Mucho gusto', 'Pizarrón', 'Por favor', 'Pregunta', 'Profesor', 'Responder', 'Terminar']

Distribución de clases:
  Clase 0 (Adiós): 76 muestras
  Clase 1 (Buenas noches): 10 muestras
  Clase 2 (Buenas tardes): 22 muestras
  Clase 3 (Buenos días): 22 muestras
  Clase 4 (Clase): 18 muestras
  Clase 5 (Comenzar): 18 muestras
  Clase 6 (Compañero): 28 muestras
  Clase 7 (Cuaderno): 18 muestras
  Clase 8 (Cómo está): 22 muestras
  Clase 9 (Deberes): 16 muestras
  Clase 10 (Disculpa): 12 muestras
  Cl

In [60]:
# 2. DATASET PYTORCH
class VideoTransformerDataset(Dataset):
    def __init__(self, X, y, masks):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        self.masks = torch.BoolTensor(masks)  # True = válido, False = padding
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return {
            'sequence': self.X[idx],      # (96, 128)
            'label': self.y[idx],          # scalar
            'mask': self.masks[idx]        # (96,)
        }

# Train-test split (80-20)
X_train, X_test, y_train, y_test, masks_train, masks_test = train_test_split(
    X, y, masks, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val, masks_train, masks_val = train_test_split(
    X_train, y_train, masks_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# DataLoaders
batch_size = 8
train_dataset = VideoTransformerDataset(X_train, y_train, masks_train)
val_dataset = VideoTransformerDataset(X_val, y_val, masks_val)
test_dataset = VideoTransformerDataset(X_test, y_test, masks_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Batches - Train: {len(train_loader)}, Val: {len(val_loader)}, Test: {len(test_loader)}")

Train: (552, 96, 300), Val: (139, 96, 300), Test: (173, 96, 300)
Batches - Train: 69, Val: 18, Test: 22


In [61]:
# 3. ARCHITECTURE: TRANSFORMER ENCODER-ONLY
class LearnablePositionalEncoding(nn.Module):
    """Positional encoding aprendible"""
    def __init__(self, d_model, max_len=96):
        super().__init__()
        self.pe = nn.Parameter(torch.randn(1, max_len, d_model))
        nn.init.normal_(self.pe, mean=0, std=0.02)
    
    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

class TransformerEncoderOnlyClassifier(nn.Module):
    """
    Transformer Encoder-Only para clasificación de secuencias temporales
    - NO usa decoder
    - Usa masked mean pooling
    - Clasificación global por secuencia
    """
    def __init__(
        self,
        input_dim=128,
        d_model=384,
        num_heads=4,
        num_layers=6,
        dim_feedforward=512,
        dropout=0.15,
        num_classes=30,
        max_seq_len=96,
        mlp_dropout=0.3,
        activation='gelu'
    ):
        super().__init__()
        
        self.d_model = d_model
        
        # 1. Proyección inicial (128 → 256)
        self.input_projection = nn.Linear(input_dim, d_model)
        
        # 2. Positional Encoding aprendible
        self.pos_encoding = LearnablePositionalEncoding(d_model, max_seq_len)
        self.dropout = nn.Dropout(dropout)
        
        # 3. Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation=activation,
            batch_first=True,
            norm_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
            norm=nn.LayerNorm(d_model)
        )
        
        # 4. Classification Head (MLP: 384 → 192 → num_classes)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 192),
            nn.GELU(),
            nn.Dropout(mlp_dropout),
            nn.Linear(192, num_classes)
        )
    
    def forward(self, src, src_key_padding_mask=None):
        """
        Args:
            src: (batch_size, seq_len, input_dim) = (B, 96, 128)
            src_key_padding_mask: (batch_size, seq_len) = True para padding
        Returns:
            logits: (batch_size, num_classes)
        """
        # 1. Proyección inicial
        x = self.input_projection(src)  # (B, 96, 256)
        
        # 2. Positional encoding
        x = self.pos_encoding(x)
        x = self.dropout(x)
        
        # 3. Transformer encoder con máscara
        x = self.transformer_encoder(
            x,
            src_key_padding_mask=src_key_padding_mask
        )  # (B, 96, 256)
        
        # 4. Masked mean pooling (solo frames válidos)
        if src_key_padding_mask is not None:
            # src_key_padding_mask: True = padding (ignorar)
            # Convertir a float: 0 para padding, 1 para válido
            mask_float = (~src_key_padding_mask).float().unsqueeze(-1)  # (B, 96, 1)
            x_masked = x * mask_float  # (B, 96, 256)
            sum_masked = x_masked.sum(dim=1)  # (B, 256)
            count_valid = mask_float.sum(dim=1)  # (B, 1)
            x_pooled = sum_masked / (count_valid + 1e-9)  # (B, 256)
        else:
            # Sin máscara: mean pooling simple
            x_pooled = x.mean(dim=1)  # (B, 256)
        
        # 5. Clasificador
        logits = self.classifier(x_pooled)  # (B, num_classes)
        
        return logits

# Crear modelo
num_classes = len(np.unique(y))
model = TransformerEncoderOnlyClassifier(
    input_dim=300,  # UMAP embeddings
    d_model=384,
    num_heads=4,
    num_layers=6,
    dim_feedforward=512,
    dropout=0.15,
    num_classes=num_classes,
    max_seq_len=96,
    mlp_dropout=0.3,
    activation='gelu'
).to(device)

# Contar parámetros
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nModelo:")
print(f"Total params: {total_params:,}")
print(f"Trainable params: {trainable_params:,}")
print(model)


Modelo:
Total params: 6,154,974
Trainable params: 6,154,974
TransformerEncoderOnlyClassifier(
  (input_projection): Linear(in_features=300, out_features=384, bias=True)
  (pos_encoding): LearnablePositionalEncoding()
  (dropout): Dropout(p=0.15, inplace=False)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
        )
        (linear1): Linear(in_features=384, out_features=512, bias=True)
        (dropout): Dropout(p=0.15, inplace=False)
        (linear2): Linear(in_features=512, out_features=384, bias=True)
        (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.15, inplace=False)
        (dropout2): Dropout(p=0.15, inplace=False)
      )
    )
    (norm): LayerNorm(

In [ ]:
# 4. ENTRENAMIENTO - CONFIG
config = {
    'optimizer': 'AdamW',
    'lr': 1e-3,
    'weight_decay': 1e-4,
    'loss': 'CrossEntropyLoss',
    'label_smoothing': 0.1,
    'batch_size': 8,
    'max_epochs': 50,
    'early_stopping_patience': 20,
    'gradient_clip': 1.0,
    'num_classes': num_classes,
    'input_dim': 300,
    'dataset_type': 'UMAP Embeddings',
    'device': str(device)
}

# Loss con label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=config['label_smoothing'])

# Optimizer
optimizer = AdamW(
    model.parameters(),
    lr=config['lr'],
    weight_decay=config['weight_decay']
)

# LR Scheduler con warm-up
from torch.optim.lr_scheduler import LinearLR, ExponentialLR, SequentialLR

# Warm-up: aumentar LR linealmente en primeros 5 epochs
warmup_scheduler = LinearLR(
    optimizer, 
    start_factor=0.1, 
    total_iters=5
)

# Decay: reducir LR exponencialmente después del warm-up (5% por época)
decay_scheduler = ExponentialLR(
    optimizer,
    gamma=0.95
)

scheduler = SequentialLR(
    optimizer,
    schedulers=[warmup_scheduler, decay_scheduler],
    milestones=[5]
)

print("Configuración de entrenamiento:")
for k, v in config.items():
    print(f"  {k}: {v}")

ValueError: SequentialLR does not support `ReduceLROnPlateau` scheduler as it requires additional kwargs to be specified when calling `step`, but got one at index 1 in the given schedulers sequence.

In [53]:
# 5. FUNCIONES DE ENTRENAMIENTO Y EVALUACIÓN
def train_epoch(model, loader, criterion, optimizer, device, grad_clip=1.0):
    model.train()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    
    for batch in tqdm(loader, desc="Train", leave=False):
        sequences = batch['sequence'].to(device)  # (B, 96, 128)
        labels = batch['label'].to(device)        # (B,)
        masks = batch['mask'].to(device)          # (B, 96)
        
        # Forward
        logits = model(sequences, src_key_padding_mask=~masks)
        loss = criterion(logits, labels)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        
        optimizer.step()
        
        total_loss += loss.item()
        all_preds.extend(logits.argmax(dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = total_loss / len(loader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    return epoch_loss, epoch_acc

@torch.no_grad()
def eval_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    all_logits = []
    
    for batch in tqdm(loader, desc="Eval", leave=False):
        sequences = batch['sequence'].to(device)
        labels = batch['label'].to(device)
        masks = batch['mask'].to(device)
        
        logits = model(sequences, src_key_padding_mask=~masks)
        loss = criterion(logits, labels)
        
        total_loss += loss.item()
        all_preds.extend(logits.argmax(dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_logits.extend(logits.cpu().numpy())
    
    epoch_loss = total_loss / len(loader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    return epoch_loss, epoch_acc, np.array(all_preds), np.array(all_labels), np.array(all_logits)

print("Funciones de entrenamiento definidas ✓")

Funciones de entrenamiento definidas ✓


In [54]:
# 6. ENTRENAMIENTO PRINCIPAL
training_log = {
    'epoch': [],
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'lr': []
}

best_val_acc = 0.0
best_epoch = 0
patience_counter = 0
max_epochs = config['max_epochs']
early_stopping_patience = config['early_stopping_patience']

# Crear directorio para guardar modelos
Path('./g5.0_umap').mkdir(exist_ok=True)

print(f"\n{'='*80}")
print(f"Iniciando entrenamiento - Epoch max: {max_epochs}, Patience: {early_stopping_patience}")
print(f"{'='*80}\n")

for epoch in range(max_epochs):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Val
    val_loss, val_acc, _, _, _ = eval_epoch(model, val_loader, criterion, device)
    
    # LR Scheduler
    current_lr = optimizer.param_groups[0]['lr']
    
    # Actualizar scheduler según la etapa
    if epoch < 5:
        # Warm-up: usar LinearLR
        warmup_scheduler.step()
    else:
        # Plateau reduction: usar ReduceLROnPlateau
        main_scheduler.step(val_acc)
    
    # Log
    training_log['epoch'].append(epoch)
    training_log['train_loss'].append(train_loss)
    training_log['train_acc'].append(train_acc)
    training_log['val_loss'].append(val_loss)
    training_log['val_acc'].append(val_acc)
    training_log['lr'].append(current_lr)
    
    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch
        patience_counter = 0
        # Guardar mejor modelo
        best_model_path = Path('./g5.0_umap/best_model.pt')
        torch.save(model.state_dict(), best_model_path)
    else:
        patience_counter += 1
    
    # Print
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}/{max_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | LR: {current_lr:.2e}")
    
    # Early stopping trigger
    if patience_counter >= early_stopping_patience:
        print(f"\nEarly stopping at epoch {epoch+1}")
        break

# Cargar mejor modelo
model.load_state_dict(torch.load(best_model_path, map_location=device))
print(f"\nMejor modelo cargado desde epoch {best_epoch} con Val Acc: {best_val_acc:.4f}")


Iniciando entrenamiento - Epoch max: 50, Patience: 20



Epoch   1/50 | Train Loss: 3.0503 | Train Acc: 0.1395 | Val Loss: 3.1043 | Val Acc: 0.1223 | LR: 5.00e-04


Epoch   5/50 | Train Loss: 2.9633 | Train Acc: 0.1649 | Val Loss: 3.0261 | Val Acc: 0.1007 | LR: 5.00e-04


Epoch  10/50 | Train Loss: 2.9410 | Train Acc: 0.1793 | Val Loss: 2.8956 | Val Acc: 0.1439 | LR: 5.00e-04


Epoch  15/50 | Train Loss: 2.8930 | Train Acc: 0.1957 | Val Loss: 2.8535 | Val Acc: 0.2446 | LR: 5.00e-04


Epoch  20/50 | Train Loss: 2.8473 | Train Acc: 0.1920 | Val Loss: 2.7346 | Val Acc: 0.2230 | LR: 5.00e-04


Epoch  25/50 | Train Loss: 2.8111 | Train Acc: 0.2101 | Val Loss: 3.0222 | Val Acc: 0.1727 | LR: 4.99e-04


Epoch  30/50 | Train Loss: 2.7258 | Train Acc: 0.2228 | Val Loss: 2.6868 | Val Acc: 0.2014 | LR: 4.99e-04


Epoch  35/50 | Train Loss: 2.6879 | Train Acc: 0.2337 | Val Loss: 2.8014 | Val Acc: 0.1367 | LR: 4.99e-04

Early stopping at epoch 35

Mejor modelo cargado desde epoch 14 con Val Acc: 0.2446


In [16]:
# 7. EVALUACIÓN EN TEST SET
# Reimport sklearn metrics (caso se hayan sobreescrito)
from sklearn.metrics import f1_score, precision_score, recall_score

print("Evaluando en Test Set...")
test_loss, test_acc, test_preds, test_labels, test_logits = eval_epoch(
    model, test_loader, criterion, device
)

print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Métricas adicionales
macro_f1 = f1_score(test_labels, test_preds, average='macro', zero_division=0)
macro_precision = precision_score(test_labels, test_preds, average='macro', zero_division=0)
macro_recall = recall_score(test_labels, test_preds, average='macro', zero_division=0)
top3_acc = top_k_accuracy_score(test_labels, test_logits, k=3, labels=np.arange(num_classes))

print(f"Macro-F1: {macro_f1:.4f}")
print(f"Macro-Precision: {macro_precision:.4f}")
print(f"Macro-Recall: {macro_recall:.4f}")
print(f"Top-3 Accuracy: {top3_acc:.4f}")

# F1 por clase
f1_per_class = f1_score(test_labels, test_preds, average=None, zero_division=0)
print(f"\nF1 Score por clase (primeras 5 clases):")
for i in range(min(5, num_classes)):
    print(f"  Clase {i}: {f1_per_class[i]:.4f}")

# Matriz de confusión
cm = confusion_matrix(test_labels, test_preds)
print(f"\nMatriz de confusión shape: {cm.shape}")

Evaluando en Test Set...



Test Loss: 2.5841
Test Accuracy: 0.2659
Macro-F1: 0.0920
Macro-Precision: 0.0776
Macro-Recall: 0.1233
Top-3 Accuracy: 0.5318

F1 Score por clase (primeras 5 clases):
  Clase 0: 0.2759
  Clase 1: 0.0000
  Clase 2: 0.0000
  Clase 3: 0.0000
  Clase 4: 0.0000

Matriz de confusión shape: (30, 30)


In [17]:
# 8. GUARDAR RESULTADOS BASELINE
# Crear directorio de salida
output_dir.mkdir(parents=True, exist_ok=True)
print(f"\n💾 Guardando resultados en {output_dir}...")

# 1. Training Log CSV
pd.DataFrame(training_log).to_csv(output_dir / 'training_log.csv', index=False)
print(f"✓ Guardado: training_log.csv")

# 2. Metrics CSV
pd.DataFrame({
    'Metric': ['Accuracy', 'Macro-F1', 'Macro-Precision', 'Macro-Recall', 'Top-3 Accuracy', 'Test Loss'],
    'Value': [test_acc, macro_f1, macro_precision, macro_recall, top3_acc, test_loss]
}).to_csv(output_dir / 'metrics.csv', index=False)
print(f"✓ Guardado: metrics.csv")

# 3. Per-class metrics
# Extract unique class names (one per class ID)
unique_class_names = []
for class_id in range(num_classes):
    idx = np.where(y == class_id)[0][0]
    # Extract base name without extension
    class_name = str(filenames[idx]).replace('.json', '').split('_')[0]
    unique_class_names.append(class_name)

class_report = classification_report(
    test_labels, test_preds, 
    target_names=unique_class_names,
    output_dict=True, zero_division=0
)
pd.DataFrame(class_report).T.to_csv(output_dir / 'per_class_metrics.csv')
print(f"✓ Guardado: per_class_metrics.csv")

# 4. Confusion Matrix CSV
pd.DataFrame(cm).to_csv(output_dir / 'confusion_matrix.csv', index=False, header=False)
print(f"✓ Guardado: confusion_matrix.csv")

# 5. Config JSON
model_config = {
    'experiment_type': 'baseline',
    'architecture': 'TransformerEncoderOnly',
    'input_dim': config['input_dim'],
    'd_model': 256,
    'num_heads': 8,
    'num_layers': 6,
    'dim_feedforward': 1024,
    'dropout': current_config['dropout'],
    'num_classes': num_classes,
    'max_seq_len': 96,
    'use_class_weights': current_config['use_class_weights'],
    'label_smoothing': current_config['label_smoothing'],
    'best_epoch': int(best_epoch),
    'best_val_acc': float(best_val_acc),
    'test_accuracy': float(test_acc),
    'test_macro_f1': float(macro_f1),
    'training_timestamp': datetime.now().isoformat()
}
with open(output_dir / 'config.json', 'w', encoding='utf-8') as f:
    json.dump(model_config, f, indent=2, ensure_ascii=False)
print(f"✓ Guardado: config.json")


💾 Guardando resultados en C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS...
✓ Guardado: training_log.csv
✓ Guardado: metrics.csv
✓ Guardado: per_class_metrics.csv
✓ Guardado: confusion_matrix.csv
✓ Guardado: config.json


In [18]:
# 9. VISUALIZACIONES - CURVAS DE APRENDIZAJE
print(f"\n🎨 Generando visualizaciones...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].plot(training_log['epoch'], training_log['train_loss'], 'b-', label='Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(training_log['epoch'], training_log['val_loss'], 'r-', label='Val Loss', marker='s', linewidth=2)
axes[0, 0].axvline(best_epoch, color='g', linestyle='--', alpha=0.7, linewidth=2, label=f'Best Epoch {best_epoch+1}')
axes[0, 0].set_xlabel('Época', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Curva de Loss', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(training_log['epoch'], training_log['train_acc'], 'b-', label='Train Acc', marker='o', linewidth=2)
axes[0, 1].plot(training_log['epoch'], training_log['val_acc'], 'r-', label='Val Acc', marker='s', linewidth=2)
axes[0, 1].axvline(best_epoch, color='g', linestyle='--', alpha=0.7, linewidth=2, label=f'Best Epoch {best_epoch+1}')
axes[0, 1].set_xlabel('Época', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Curva de Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(training_log['epoch'], training_log['lr'], 'g-', marker='o', linewidth=2)
axes[1, 0].set_xlabel('Época', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Programación de Learning Rate', fontsize=14, fontweight='bold')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(True, alpha=0.3)

metrics_names = ['Accuracy', 'Macro-F1', 'Top-3 Acc']
metrics_values = [test_acc, macro_f1, top3_acc]
colors = ['#3498db', '#e74c3c', '#2ecc71']
bars = axes[1, 1].bar(metrics_names, metrics_values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
axes[1, 1].set_ylabel('Score', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Métricas en Test Set', fontsize=14, fontweight='bold')
axes[1, 1].set_ylim([0, 1.05])
for bar, v in zip(bars, metrics_values):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02, 
                    f'{v:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.suptitle(f'Curvas de Aprendizaje - {EXPERIMENT_TYPE.upper()}', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()

plot_path = output_dir / 'training_curves.png'
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"✓ Guardado: {plot_path}")
plt.close()

print("✓ Curvas de aprendizaje generadas")


🎨 Generando visualizaciones...
✓ Guardado: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS\training_curves.png
✓ Curvas de aprendizaje generadas


In [19]:
# 10. MATRIZ DE CONFUSIÓN CON NOMBRES
print(f"\n🎨 Generando matriz de confusión...")

unique_classes_list = sorted(list(set(test_labels)))
class_labels = [filenames[i] if i < len(filenames) else f'Class {i}' for i in unique_classes_list]

fig, ax = plt.subplots(figsize=(20, 18))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            cbar_kws={'label': 'Muestras'}, ax=ax,
            xticklabels=class_labels, yticklabels=class_labels, 
            square=True)
ax.set_xlabel('Clase Predicha', fontsize=14, fontweight='bold')
ax.set_ylabel('Clase Real', fontsize=14, fontweight='bold')
ax.set_title(f'Matriz de Confusión - {EXPERIMENT_TYPE.upper()} (Accuracy: {test_acc:.4f})', 
             fontsize=16, fontweight='bold')
plt.tight_layout()

cm_plot_path = output_dir / 'confusion_matrix.png'
plt.savefig(cm_plot_path, dpi=300, bbox_inches='tight')
print(f"✓ Guardado: {cm_plot_path}")
plt.close()

print("✓ Matriz de confusión completada")


🎨 Generando matriz de confusión...
✓ Guardado: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS\confusion_matrix.png
✓ Matriz de confusión completada


In [20]:
# 11. ANÁLISIS POR CLASE
print(f"\n🎨 Generando análisis por clase...")

# Calcular precision, recall, f1 por clase
precision_per_class = []
recall_per_class = []
f1_per_class_array = []

for i in range(num_classes):
    if i in unique_classes_list:
        idx = unique_classes_list.index(i)
        tp = cm[idx, idx]
        fp = cm[:, idx].sum() - tp
        fn = cm[idx, :].sum() - tp
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_value = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        precision_per_class.append(precision)
        recall_per_class.append(recall)
        f1_per_class_array.append(f1_value)
    else:
        precision_per_class.append(0)
        recall_per_class.append(0)
        f1_per_class_array.append(0)

precision_per_class = np.array(precision_per_class)
recall_per_class = np.array(recall_per_class)
f1_per_class_array = np.array(f1_per_class_array)

# Visualización
fig_pc, axes_pc = plt.subplots(1, 3, figsize=(24, 8))

y_pos = np.arange(num_classes)
axes_pc[0].barh(y_pos, precision_per_class, color='skyblue', edgecolor='navy', alpha=0.7)
axes_pc[0].set_yticks(y_pos)
axes_pc[0].set_yticklabels([filenames[i] if i < len(filenames) else f'C{i}' for i in range(num_classes)], fontsize=8)
axes_pc[0].set_xlabel('Precision', fontsize=12, fontweight='bold')
axes_pc[0].set_title('Precision por Clase', fontsize=14, fontweight='bold')
axes_pc[0].set_xlim([0, 1])
axes_pc[0].grid(True, alpha=0.3, axis='x')

axes_pc[1].barh(y_pos, recall_per_class, color='lightcoral', edgecolor='darkred', alpha=0.7)
axes_pc[1].set_yticks(y_pos)
axes_pc[1].set_yticklabels([filenames[i] if i < len(filenames) else f'C{i}' for i in range(num_classes)], fontsize=8)
axes_pc[1].set_xlabel('Recall', fontsize=12, fontweight='bold')
axes_pc[1].set_title('Recall por Clase', fontsize=14, fontweight='bold')
axes_pc[1].set_xlim([0, 1])
axes_pc[1].grid(True, alpha=0.3, axis='x')

axes_pc[2].barh(y_pos, f1_per_class_array, color='lightgreen', edgecolor='darkgreen', alpha=0.7)
axes_pc[2].set_yticks(y_pos)
axes_pc[2].set_yticklabels([filenames[i] if i < len(filenames) else f'C{i}' for i in range(num_classes)], fontsize=8)
axes_pc[2].set_xlabel('F1-Score', fontsize=12, fontweight='bold')
axes_pc[2].set_title('F1-Score por Clase', fontsize=14, fontweight='bold')
axes_pc[2].set_xlim([0, 1])
axes_pc[2].grid(True, alpha=0.3, axis='x')

plt.suptitle(f'Análisis por Clase - {EXPERIMENT_TYPE.upper()}', fontsize=16, fontweight='bold')
plt.tight_layout()

per_class_plot_path = output_dir / 'per_class_analysis.png'
plt.savefig(per_class_plot_path, dpi=300, bbox_inches='tight')
print(f"✓ Guardado: {per_class_plot_path}")
plt.close()

print("✓ Análisis por clase completado")


🎨 Generando análisis por clase...
✓ Guardado: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS\per_class_analysis.png
✓ Análisis por clase completado


In [21]:
# 12. RESUMEN EJECUTIVO
print(f"\n📝 Generando resumen ejecutivo...")

# Top 5 clases por F1
f1_per_class_list = [(i, f1_value) for i, f1_value in enumerate(f1_per_class_array)]
class_f1_sorted = sorted(f1_per_class_list, key=lambda x: x[1], reverse=True)

resumen = f"""
{'='*80}
RESUMEN EJECUTIVO - {current_config['folder_name']}
{'='*80}

📊 PERFORMANCE:
  • Test Accuracy:       {test_acc:.4f}
  • Macro F1-Score:      {macro_f1:.4f}
  • Macro Precision:     {macro_precision:.4f}
  • Macro Recall:        {macro_recall:.4f}
  • Top-3 Accuracy:      {top3_acc:.4f}

🏗️ ARQUITECTURA:
  • Modelo:              Transformer Encoder-Only
  • Dataset:             UMAP Embeddings (reducción de dimensionalidad)
  • Input Features:      {config['input_dim']}
  • Sequence Length:     96 frames
  • Embedding Dim:       256
  • Attention Heads:     8
  • Encoder Layers:      6
  • Dim Feedforward:     1024
  • Total Parameters:    {total_params:,}

⚙️ CONFIGURACIÓN:
  • Dropout:             {current_config['dropout']}
  • Class Weights:       {'Activado' if current_config['use_class_weights'] else 'Desactivado'}
  • Label Smoothing:     {current_config['label_smoothing']}
  • Best Epoch:          {best_epoch}
  • Optimizer:           AdamW (lr={config['lr']}, weight_decay={config['weight_decay']})
  • Scheduler:           CosineAnnealingWarmRestarts

🎯 VENTAJAS UMAP EMBEDDINGS:
  • Reducción de dimensionalidad: Preserva estructura local y global
  • Menor carga computacional
  • Máscaras por muestra nativas (más preciso)
  • Mejor generalización con menos features ruidosas

📈 TOP 5 CLASES (F1-Score):
"""
for rank, (class_idx, f1_value) in enumerate(class_f1_sorted[:5], 1):
    class_name = filenames[class_idx] if class_idx < len(filenames) else f'Class {class_idx}'
    resumen += f"  {rank}. {class_name:20s} | F1: {f1_value:.4f}\n"

resumen += f"\n{'='*80}\n"

# Guardar
resumen_path = output_dir / 'RESUMEN.txt'
with open(resumen_path, 'w', encoding='utf-8') as f:
    f.write(resumen)
print(f"✓ Guardado: {resumen_path}")

print(f"\n{'='*80}")
print(f"✅ BASELINE ({current_config['folder_name']}) COMPLETADO")
print(f"{'='*80}")
print(f"  Test Accuracy: {test_acc:.4f}")
print(f"  Macro F1:      {macro_f1:.4f}")
print(f"  Top-3 Acc:     {top3_acc:.4f}")
print(f"{'='*80}")


📝 Generando resumen ejecutivo...
✓ Guardado: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS\RESUMEN.txt

✅ BASELINE (G4-RESULTS) COMPLETADO
  Test Accuracy: 0.2659
  Macro F1:      0.0920
  Top-3 Acc:     0.5318


# Experimentos de Mejora - Transformer Encoder-Only con UMAP Embeddings

## 🎯 Objetivo
Implementar y comparar mejoras controladas sobre el modelo base para mejorar Macro-F1 y generalización usando embeddings UMAP.

### Experimentos:
- **Exp 0 (G5.0_UMAP)**: Baseline - Dropout 0.1, sin class weights, sin label smoothing
- **Exp 1 (G6_UMAP)**: Class Weights + Dropout 0.3, sin label smoothing
- **Exp 2 (G7_UMAP)**: Dropout 0.3 + Label Smoothing 0.1, sin class weights

In [22]:
# Función para crear modelo con dropout configurable
def create_model(dropout_config=0.1):
    """Crea modelo con configuración específica de dropout"""
    model = TransformerEncoderOnlyClassifier(
        input_dim=config['input_dim'],  # 128 para UMAP
        d_model=256,
        num_heads=8,
        num_layers=6,
        dim_feedforward=1024,
        dropout=dropout_config,
        num_classes=num_classes,
        max_seq_len=96,
        mlp_dropout=0.2,
        activation='gelu'
    ).to(device)
    return model

print("✓ Función create_model definida")

✓ Función create_model definida


## 🧪 Experimento 0 (G5.0_UMAP) - Baseline
- Dropout: 0.1
- Sin class weights
- Sin label smoothing
- Resultados ya obtenidos arriba

In [23]:
# Resultados Exp 0 (Baseline)
exp0_results = {
    'experiment': 'G4-RESULTS',
    'dropout': current_config['dropout'],
    'class_weights': current_config.get('use_class_weights', False),
    'label_smoothing': current_config.get('label_smoothing', 0.0),
    'test_accuracy': test_acc,
    'test_macro_f1': macro_f1,
    'test_top3_accuracy': top3_acc,
    'test_loss': test_loss,
    'best_epoch': best_epoch,
    'best_val_acc': best_val_acc
}

print(f"✓ Resultados {EXPERIMENT_TYPE} registrados")

✓ Resultados baseline registrados


## 🧪 Experimento 1 (G6_UMAP) - Class Weights + Dropout 0.3
- Class weights calculados de y_train
- Dropout: 0.3 (mayor regularización)
- Sin label smoothing

In [25]:
# EXPERIMENTO 1: G4-RESULTS-CLASS-WEIGHTS
print("\n" + "="*80)
print("Iniciando Experimento 1: G4-RESULTS-CLASS-WEIGHTS")
print("="*80)

# Configurar directorio de salida
output_dir_exp1 = ROOT_PATH / 'G4-RESULTS-CLASS-WEIGHTS'
output_dir_exp1.mkdir(parents=True, exist_ok=True)
print(f"📁 Directorio: {output_dir_exp1}")

# Calcular class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights_array = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.FloatTensor(class_weights_array).to(device)

# Crear modelo con dropout 0.3
model_exp1 = create_model(dropout_config=0.3)
criterion_exp1 = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.0)
optimizer_exp1 = AdamW(model_exp1.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
scheduler_exp1 = ReduceLROnPlateau(optimizer_exp1, mode='max', factor=0.5, patience=5)

# Entrenamiento
training_log_exp1 = {'epoch': [], 'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}
best_val_acc_exp1 = 0.0
best_epoch_exp1 = 0
patience_counter_exp1 = 0

print(f"\nEntrenando con Class Weights y Dropout 0.3...")
for epoch in range(config['max_epochs']):
    train_loss, train_acc = train_epoch(model_exp1, train_loader, criterion_exp1, optimizer_exp1, device)
    val_loss, val_acc, _, _, _ = eval_epoch(model_exp1, val_loader, criterion_exp1, device)
    
    current_lr = optimizer_exp1.param_groups[0]['lr']
    scheduler_exp1.step(val_acc)
    
    training_log_exp1['epoch'].append(epoch)
    training_log_exp1['train_loss'].append(train_loss)
    training_log_exp1['train_acc'].append(train_acc)
    training_log_exp1['val_loss'].append(val_loss)
    training_log_exp1['val_acc'].append(val_acc)
    training_log_exp1['lr'].append(current_lr)
    
    if val_acc > best_val_acc_exp1:
        best_val_acc_exp1 = val_acc
        best_epoch_exp1 = epoch
        patience_counter_exp1 = 0
        torch.save(model_exp1.state_dict(), output_dir_exp1 / 'best_model.pt')
    else:
        patience_counter_exp1 += 1
    
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}/{config['max_epochs']} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    if patience_counter_exp1 >= config['early_stopping_patience']:
        print(f"Early stopping at epoch {epoch+1}")
        break

# Cargar mejor modelo
model_exp1.load_state_dict(torch.load(output_dir_exp1 / 'best_model.pt', map_location=device))

# Reimport sklearn metrics (caso se hayan sobreescrito)
from sklearn.metrics import f1_score, precision_score, recall_score

# Evaluación
test_loss_exp1, test_acc_exp1, test_preds_exp1, test_labels_exp1, test_logits_exp1 = eval_epoch(
    model_exp1, test_loader, criterion_exp1, device
)

macro_f1_exp1 = f1_score(test_labels_exp1, test_preds_exp1, average='macro', zero_division=0)
macro_precision_exp1 = precision_score(test_labels_exp1, test_preds_exp1, average='macro', zero_division=0)
macro_recall_exp1 = recall_score(test_labels_exp1, test_preds_exp1, average='macro', zero_division=0)
top3_acc_exp1 = top_k_accuracy_score(test_labels_exp1, test_logits_exp1, k=3, labels=np.arange(num_classes))

print(f"\n✓ CLASS-WEIGHTS completado:")
print(f"  Test Accuracy: {test_acc_exp1:.4f}")
print(f"  Macro F1: {macro_f1_exp1:.4f}")
print(f"  Top-3 Acc: {top3_acc_exp1:.4f}")

# 1. Training log
pd.DataFrame(training_log_exp1).to_csv(output_dir_exp1 / 'training_log.csv', index=False)
print(f"✓ Guardado: training_log.csv")

# 2. Metrics CSV
pd.DataFrame({
    'Metric': ['Accuracy', 'Macro-F1', 'Macro-Precision', 'Macro-Recall', 'Top-3 Accuracy', 'Test Loss'],
    'Value': [test_acc_exp1, macro_f1_exp1, macro_precision_exp1, macro_recall_exp1, top3_acc_exp1, test_loss_exp1]
}).to_csv(output_dir_exp1 / 'metrics.csv', index=False)
print(f"✓ Guardado: metrics.csv")

# 3. Per-class metrics
# Extract unique class names (one per class ID)
unique_class_names_exp1 = []
for class_id in range(num_classes):
    idx = np.where(y == class_id)[0][0]
    class_name = str(filenames[idx]).replace('.json', '').split('_')[0]
    unique_class_names_exp1.append(class_name)

class_report_dict = classification_report(
    test_labels_exp1, test_preds_exp1, 
    target_names=unique_class_names_exp1,
    output_dict=True, zero_division=0
)
pd.DataFrame(class_report_dict).T.to_csv(output_dir_exp1 / 'per_class_metrics.csv')
print(f"✓ Guardado: per_class_metrics.csv")

# 4. Confusion matrix CSV
cm_exp1 = confusion_matrix(test_labels_exp1, test_preds_exp1)
pd.DataFrame(cm_exp1).to_csv(output_dir_exp1 / 'confusion_matrix.csv', index=False, header=False)
print(f"✓ Guardado: confusion_matrix.csv")

# 5. Config JSON
config_exp1 = {
    'experiment_type': 'class_weights',
    'architecture': 'TransformerEncoderOnly',
    'input_dim': config['input_dim'],
    'd_model': 256,
    'num_heads': 8,
    'num_layers': 6,
    'dim_feedforward': 1024,
    'dropout': 0.3,
    'num_classes': num_classes,
    'max_seq_len': 96,
    'use_class_weights': True,
    'label_smoothing': 0.0,
    'best_epoch': int(best_epoch_exp1),
    'best_val_acc': float(best_val_acc_exp1),
    'test_accuracy': float(test_acc_exp1),
    'test_macro_f1': float(macro_f1_exp1),
    'training_timestamp': datetime.now().isoformat()
}
with open(output_dir_exp1 / 'config.json', 'w', encoding='utf-8') as f:
    json.dump(config_exp1, f, indent=2, ensure_ascii=False)
print(f"✓ Guardado: config.json")

# 6. Training curves PNG
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].plot(training_log_exp1['epoch'], training_log_exp1['train_loss'], 'b-', label='Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(training_log_exp1['epoch'], training_log_exp1['val_loss'], 'r-', label='Val Loss', marker='s', linewidth=2)
axes[0, 0].axvline(best_epoch_exp1, color='g', linestyle='--', alpha=0.7, linewidth=2, label=f'Best Epoch {best_epoch_exp1+1}')
axes[0, 0].set_xlabel('Época', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Curva de Loss', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(training_log_exp1['epoch'], training_log_exp1['train_acc'], 'b-', label='Train Acc', marker='o', linewidth=2)
axes[0, 1].plot(training_log_exp1['epoch'], training_log_exp1['val_acc'], 'r-', label='Val Acc', marker='s', linewidth=2)
axes[0, 1].axvline(best_epoch_exp1, color='g', linestyle='--', alpha=0.7, linewidth=2, label=f'Best Epoch {best_epoch_exp1+1}')
axes[0, 1].set_xlabel('Época', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Curva de Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(training_log_exp1['epoch'], training_log_exp1['lr'], 'g-', marker='o', linewidth=2)
axes[1, 0].set_xlabel('Época', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Learning Rate', fontsize=14, fontweight='bold')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(True, alpha=0.3)

metrics_names = ['Accuracy', 'Macro-F1', 'Top-3 Acc']
metrics_values = [test_acc_exp1, macro_f1_exp1, top3_acc_exp1]
colors = ['#3498db', '#e74c3c', '#2ecc71']
bars = axes[1, 1].bar(metrics_names, metrics_values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
axes[1, 1].set_ylabel('Score', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Métricas en Test Set', fontsize=14, fontweight='bold')
axes[1, 1].set_ylim([0, 1.05])
for bar, v in zip(bars, metrics_values):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02, 
                    f'{v:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Curvas de Aprendizaje - CLASS-WEIGHTS', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(output_dir_exp1 / 'training_curves.png', dpi=300, bbox_inches='tight')
print(f"✓ Guardado: training_curves.png")
plt.close()

# 7. Confusion matrix PNG
unique_classes_exp1 = sorted(list(set(test_labels_exp1)))
class_labels_exp1 = [filenames[i] if i < len(filenames) else f'Class {i}' for i in unique_classes_exp1]

fig_cm, ax_cm = plt.subplots(figsize=(20, 18))
sns.heatmap(cm_exp1, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Muestras'}, ax=ax_cm,
            xticklabels=class_labels_exp1, yticklabels=class_labels_exp1, square=True)
ax_cm.set_xlabel('Clase Predicha', fontsize=14, fontweight='bold')
ax_cm.set_ylabel('Clase Real', fontsize=14, fontweight='bold')
ax_cm.set_title(f'Matriz de Confusión - CLASS-WEIGHTS (Accuracy: {test_acc_exp1:.4f})', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(output_dir_exp1 / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
print(f"✓ Guardado: confusion_matrix.png")
plt.close()

# 8. Per-class analysis PNG
precision_per_class_exp1 = []
recall_per_class_exp1 = []
f1_per_class_array_exp1 = []

for i in range(num_classes):
    if i in unique_classes_exp1:
        idx = unique_classes_exp1.index(i)
        tp = cm_exp1[idx, idx]
        fp = cm_exp1[:, idx].sum() - tp
        fn = cm_exp1[idx, :].sum() - tp
        
        prec = tp / (tp + fp) if (tp + fp) > 0 else 0
        rec = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_value = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
        
        precision_per_class_exp1.append(prec)
        recall_per_class_exp1.append(rec)
        f1_per_class_array_exp1.append(f1_value)
    else:
        precision_per_class_exp1.append(0)
        recall_per_class_exp1.append(0)
        f1_per_class_array_exp1.append(0)

precision_per_class_exp1 = np.array(precision_per_class_exp1)
recall_per_class_exp1 = np.array(recall_per_class_exp1)
f1_per_class_array_exp1 = np.array(f1_per_class_array_exp1)

fig_pc, axes_pc = plt.subplots(1, 3, figsize=(24, 8))

y_pos = np.arange(num_classes)
axes_pc[0].barh(y_pos, precision_per_class_exp1, color='skyblue', edgecolor='navy', alpha=0.7)
axes_pc[0].set_yticks(y_pos)
axes_pc[0].set_yticklabels([filenames[i] if i < len(filenames) else f'C{i}' for i in range(num_classes)], fontsize=8)
axes_pc[0].set_xlabel('Precision', fontsize=12, fontweight='bold')
axes_pc[0].set_title('Precision por Clase', fontsize=14, fontweight='bold')
axes_pc[0].set_xlim([0, 1])
axes_pc[0].grid(True, alpha=0.3, axis='x')

axes_pc[1].barh(y_pos, recall_per_class_exp1, color='lightcoral', edgecolor='darkred', alpha=0.7)
axes_pc[1].set_yticks(y_pos)
axes_pc[1].set_yticklabels([filenames[i] if i < len(filenames) else f'C{i}' for i in range(num_classes)], fontsize=8)
axes_pc[1].set_xlabel('Recall', fontsize=12, fontweight='bold')
axes_pc[1].set_title('Recall por Clase', fontsize=14, fontweight='bold')
axes_pc[1].set_xlim([0, 1])
axes_pc[1].grid(True, alpha=0.3, axis='x')

axes_pc[2].barh(y_pos, f1_per_class_array_exp1, color='lightgreen', edgecolor='darkgreen', alpha=0.7)
axes_pc[2].set_yticks(y_pos)
axes_pc[2].set_yticklabels([filenames[i] if i < len(filenames) else f'C{i}' for i in range(num_classes)], fontsize=8)
axes_pc[2].set_xlabel('F1-Score', fontsize=12, fontweight='bold')
axes_pc[2].set_title('F1-Score por Clase', fontsize=14, fontweight='bold')
axes_pc[2].set_xlim([0, 1])
axes_pc[2].grid(True, alpha=0.3, axis='x')

plt.suptitle('Análisis por Clase - CLASS-WEIGHTS', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(output_dir_exp1 / 'per_class_analysis.png', dpi=300, bbox_inches='tight')
print(f"✓ Guardado: per_class_analysis.png")
plt.close()

# 9. RESUMEN.txt
f1_per_class_list_exp1 = [(i, f1_value) for i, f1_value in enumerate(f1_per_class_array_exp1)]
class_f1_sorted = sorted(f1_per_class_list_exp1, key=lambda x: x[1], reverse=True)

summary_exp1 = f"""
{'='*80}
RESUMEN EJECUTIVO - G4-RESULTS-CLASS-WEIGHTS
{'='*80}

📊 PERFORMANCE:
  • Test Accuracy:       {test_acc_exp1:.4f}
  • Macro F1-Score:      {macro_f1_exp1:.4f}
  • Macro Precision:     {macro_precision_exp1:.4f}
  • Macro Recall:        {macro_recall_exp1:.4f}
  • Top-3 Accuracy:      {top3_acc_exp1:.4f}

⚙️ CONFIGURACIÓN:
  • Dropout:             0.3
  • Class Weights:       Activado (balanced)
  • Label Smoothing:     0.0
  • Best Epoch:          {best_epoch_exp1}

📈 TOP 5 CLASES (F1-Score):
"""
for rank, (class_idx, f1_value) in enumerate(class_f1_sorted[:5], 1):
    class_name = filenames[class_idx] if class_idx < len(filenames) else f'Class {class_idx}'
    summary_exp1 += f"  {rank}. {class_name:20s} | F1: {f1_value:.4f}\n"

summary_exp1 += f"\n{'='*80}\n"

with open(output_dir_exp1 / 'RESUMEN.txt', 'w', encoding='utf-8') as f:
    f.write(summary_exp1)
print(f"✓ Guardado: RESUMEN.txt")

# Resultados en diccionario
exp1_results = {
    'experiment': 'G4-RESULTS-CLASS-WEIGHTS',
    'dropout': 0.3,
    'class_weights': True,
    'label_smoothing': 0.0,
    'test_accuracy': test_acc_exp1,
    'test_macro_f1': macro_f1_exp1,
    'test_top3_accuracy': top3_acc_exp1,
    'best_epoch': best_epoch_exp1
}

print(f"{'='*80}")
print("✅ EXPERIMENTO 1 (CLASS-WEIGHTS) COMPLETADO")
print(f"\n{'='*80}")



Iniciando Experimento 1: G4-RESULTS-CLASS-WEIGHTS
📁 Directorio: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS-CLASS-WEIGHTS

Entrenando con Class Weights y Dropout 0.3...


Epoch   1/50 | Train Loss: 3.4511 | Val Acc: 0.0216


Epoch   5/50 | Train Loss: 3.4152 | Val Acc: 0.0216


Epoch  10/50 | Train Loss: 3.4105 | Val Acc: 0.1511


Epoch  15/50 | Train Loss: 3.3204 | Val Acc: 0.0863


Early stopping at epoch 18



✓ CLASS-WEIGHTS completado:
  Test Accuracy: 0.1503
  Macro F1: 0.0254
  Top-3 Acc: 0.2081
✓ Guardado: training_log.csv
✓ Guardado: metrics.csv
✓ Guardado: per_class_metrics.csv
✓ Guardado: confusion_matrix.csv
✓ Guardado: config.json
✓ Guardado: training_curves.png
✓ Guardado: confusion_matrix.png
✓ Guardado: per_class_analysis.png
✓ Guardado: RESUMEN.txt
✅ EXPERIMENTO 1 (CLASS-WEIGHTS) COMPLETADO



## 🧪 Experimento 2 (G7_UMAP) - Dropout 0.3 + Label Smoothing
- Dropout: 0.3 (mayor regularización)
- Label smoothing: 0.2 (más agresivo que baseline)
- Sin class weights

In [27]:
# EXPERIMENTO 2: G4-RESULTS-LABEL-SMOOTH
# Reimport sklearn metrics (caso se hayan sobreescrito)
from sklearn.metrics import f1_score, precision_score, recall_score

print("\n" + "="*80)
print("Iniciando Experimento 2: G4-RESULTS-LABEL-SMOOTH")
print("="*80)

# Configurar directorio
output_dir_exp2 = ROOT_PATH / 'G4-RESULTS-LABEL-SMOOTH'
output_dir_exp2.mkdir(parents=True, exist_ok=True)
print(f"📁 Directorio: {output_dir_exp2}")

# Crear modelo con dropout 0.3
model_exp2 = create_model(dropout_config=0.3)
criterion_exp2 = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer_exp2 = AdamW(model_exp2.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
scheduler_exp2 = ReduceLROnPlateau(optimizer_exp2, mode='max', factor=0.5, patience=5)

# Entrenamiento
training_log_exp2 = {'epoch': [], 'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}
best_val_acc_exp2 = 0.0
best_epoch_exp2 = 0
patience_counter_exp2 = 0

print(f"\nEntrenando con Label Smoothing 0.1 y Dropout 0.3...")
for epoch in range(config['max_epochs']):
    train_loss, train_acc = train_epoch(model_exp2, train_loader, criterion_exp2, optimizer_exp2, device)
    val_loss, val_acc, _, _, _ = eval_epoch(model_exp2, val_loader, criterion_exp2, device)
    
    current_lr = optimizer_exp2.param_groups[0]['lr']
    scheduler_exp2.step(val_acc)
    
    training_log_exp2['epoch'].append(epoch)
    training_log_exp2['train_loss'].append(train_loss)
    training_log_exp2['train_acc'].append(train_acc)
    training_log_exp2['val_loss'].append(val_loss)
    training_log_exp2['val_acc'].append(val_acc)
    training_log_exp2['lr'].append(current_lr)
    
    if val_acc > best_val_acc_exp2:
        best_val_acc_exp2 = val_acc
        best_epoch_exp2 = epoch
        patience_counter_exp2 = 0
        torch.save(model_exp2.state_dict(), output_dir_exp2 / 'best_model.pt')
    else:
        patience_counter_exp2 += 1
    
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}/{config['max_epochs']} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    if patience_counter_exp2 >= config['early_stopping_patience']:
        print(f"Early stopping at epoch {epoch+1}")
        break

# Cargar mejor modelo
model_exp2.load_state_dict(torch.load(output_dir_exp2 / 'best_model.pt', map_location=device))

# Evaluación
test_loss_exp2, test_acc_exp2, test_preds_exp2, test_labels_exp2, test_logits_exp2 = eval_epoch(
    model_exp2, test_loader, criterion_exp2, device
)

macro_f1_exp2 = f1_score(test_labels_exp2, test_preds_exp2, average='macro', zero_division=0)
macro_precision_exp2 = precision_score(test_labels_exp2, test_preds_exp2, average='macro', zero_division=0)
macro_recall_exp2 = recall_score(test_labels_exp2, test_preds_exp2, average='macro', zero_division=0)
top3_acc_exp2 = top_k_accuracy_score(test_labels_exp2, test_logits_exp2, k=3, labels=np.arange(num_classes))

print(f"\n✓ LABEL-SMOOTH completado:")
print(f"  Test Accuracy: {test_acc_exp2:.4f}")
print(f"  Macro F1: {macro_f1_exp2:.4f}")
print(f"  Top-3 Acc: {top3_acc_exp2:.4f}")

# 1. Training log
pd.DataFrame(training_log_exp2).to_csv(output_dir_exp2 / 'training_log.csv', index=False)
print(f"✓ Guardado: training_log.csv")

# 2. Metrics CSV
pd.DataFrame({
    'Metric': ['Accuracy', 'Macro-F1', 'Macro-Precision', 'Macro-Recall', 'Top-3 Accuracy', 'Test Loss'],
    'Value': [test_acc_exp2, macro_f1_exp2, macro_precision_exp2, macro_recall_exp2, top3_acc_exp2, test_loss_exp2]
}).to_csv(output_dir_exp2 / 'metrics.csv', index=False)
print(f"✓ Guardado: metrics.csv")

# 3. Per-class metrics
# Extract unique class names (one per class ID)
unique_class_names_exp2 = []
for class_id in range(num_classes):
    idx = np.where(y == class_id)[0][0]
    class_name = str(filenames[idx]).replace('.json', '').split('_')[0]
    unique_class_names_exp2.append(class_name)

class_report_dict_exp2 = classification_report(
    test_labels_exp2, test_preds_exp2, 
    target_names=unique_class_names_exp2,
    output_dict=True, zero_division=0
)
pd.DataFrame(class_report_dict_exp2).T.to_csv(output_dir_exp2 / 'per_class_metrics.csv')
print(f"✓ Guardado: per_class_metrics.csv")

# 4. Confusion matrix CSV
cm_exp2 = confusion_matrix(test_labels_exp2, test_preds_exp2)
pd.DataFrame(cm_exp2).to_csv(output_dir_exp2 / 'confusion_matrix.csv', index=False, header=False)
print(f"✓ Guardado: confusion_matrix.csv")

# 5. Config JSON
config_exp2 = {
    'experiment_type': 'label_smoothing',
    'architecture': 'TransformerEncoderOnly',
    'input_dim': config['input_dim'],
    'd_model': 256,
    'num_heads': 8,
    'num_layers': 6,
    'dim_feedforward': 1024,
    'dropout': 0.3,
    'num_classes': num_classes,
    'max_seq_len': 96,
    'use_class_weights': False,
    'label_smoothing': 0.1,
    'best_epoch': int(best_epoch_exp2),
    'best_val_acc': float(best_val_acc_exp2),
    'test_accuracy': float(test_acc_exp2),
    'test_macro_f1': float(macro_f1_exp2),
    'training_timestamp': datetime.now().isoformat()
}
with open(output_dir_exp2 / 'config.json', 'w', encoding='utf-8') as f:
    json.dump(config_exp2, f, indent=2, ensure_ascii=False)
print(f"✓ Guardado: config.json")

# 6. Training curves PNG
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].plot(training_log_exp2['epoch'], training_log_exp2['train_loss'], 'b-', label='Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(training_log_exp2['epoch'], training_log_exp2['val_loss'], 'r-', label='Val Loss', marker='s', linewidth=2)
axes[0, 0].axvline(best_epoch_exp2, color='g', linestyle='--', alpha=0.7, linewidth=2, label=f'Best Epoch {best_epoch_exp2+1}')
axes[0, 0].set_xlabel('Época', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Curva de Loss', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(training_log_exp2['epoch'], training_log_exp2['train_acc'], 'b-', label='Train Acc', marker='o', linewidth=2)
axes[0, 1].plot(training_log_exp2['epoch'], training_log_exp2['val_acc'], 'r-', label='Val Acc', marker='s', linewidth=2)
axes[0, 1].axvline(best_epoch_exp2, color='g', linestyle='--', alpha=0.7, linewidth=2, label=f'Best Epoch {best_epoch_exp2+1}')
axes[0, 1].set_xlabel('Época', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Curva de Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(training_log_exp2['epoch'], training_log_exp2['lr'], 'g-', marker='o', linewidth=2)
axes[1, 0].set_xlabel('Época', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Learning Rate', fontsize=14, fontweight='bold')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(True, alpha=0.3)

metrics_names = ['Accuracy', 'Macro-F1', 'Top-3 Acc']
metrics_values = [test_acc_exp2, macro_f1_exp2, top3_acc_exp2]
colors = ['#3498db', '#e74c3c', '#2ecc71']
bars = axes[1, 1].bar(metrics_names, metrics_values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
axes[1, 1].set_ylabel('Score', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Métricas en Test Set', fontsize=14, fontweight='bold')
axes[1, 1].set_ylim([0, 1.05])
for bar, v in zip(bars, metrics_values):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02, 
                    f'{v:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Curvas de Aprendizaje - LABEL-SMOOTH', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(output_dir_exp2 / 'training_curves.png', dpi=300, bbox_inches='tight')
print(f"✓ Guardado: training_curves.png")
plt.close()

# 7. Confusion matrix PNG
fig_cm, ax_cm = plt.subplots(figsize=(20, 18))
sns.heatmap(cm_exp2, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Muestras'}, ax=ax_cm,
            xticklabels=unique_class_names_exp2, yticklabels=unique_class_names_exp2, square=True)
ax_cm.set_xlabel('Clase Predicha', fontsize=14, fontweight='bold')
ax_cm.set_ylabel('Clase Real', fontsize=14, fontweight='bold')
ax_cm.set_title(f'Matriz de Confusión - LABEL-SMOOTH (Accuracy: {test_acc_exp2:.4f})', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(output_dir_exp2 / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
print(f"✓ Guardado: confusion_matrix.png")
plt.close()

# 8. Per-class analysis PNG
unique_classes_exp2 = sorted(list(set(test_labels_exp2)))
precision_per_class_exp2 = []
recall_per_class_exp2 = []
f1_per_class_array_exp2 = []

for i in range(num_classes):
    if i in unique_classes_exp2:
        idx = unique_classes_exp2.index(i)
        tp = cm_exp2[idx, idx]
        fp = cm_exp2[:, idx].sum() - tp
        fn = cm_exp2[idx, :].sum() - tp
        
        prec = tp / (tp + fp) if (tp + fp) > 0 else 0
        rec = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_value = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
        
        precision_per_class_exp2.append(prec)
        recall_per_class_exp2.append(rec)
        f1_per_class_array_exp2.append(f1_value)
    else:
        precision_per_class_exp2.append(0)
        recall_per_class_exp2.append(0)
        f1_per_class_array_exp2.append(0)

precision_per_class_exp2 = np.array(precision_per_class_exp2)
recall_per_class_exp2 = np.array(recall_per_class_exp2)
f1_per_class_array_exp2 = np.array(f1_per_class_array_exp2)

fig_pc, axes_pc = plt.subplots(1, 3, figsize=(24, 8))

y_pos = np.arange(num_classes)
axes_pc[0].barh(y_pos, precision_per_class_exp2, color='skyblue', edgecolor='navy', alpha=0.7)
axes_pc[0].set_yticks(y_pos)
axes_pc[0].set_yticklabels([filenames[i] if i < len(filenames) else f'C{i}' for i in range(num_classes)], fontsize=8)
axes_pc[0].set_xlabel('Precision', fontsize=12, fontweight='bold')
axes_pc[0].set_title('Precision por Clase', fontsize=14, fontweight='bold')
axes_pc[0].set_yticklabels(unique_class_names_exp2, fontsize=8)
axes_pc[0].grid(True, alpha=0.3, axis='x')

axes_pc[1].barh(y_pos, recall_per_class_exp2, color='lightcoral', edgecolor='darkred', alpha=0.7)
axes_pc[1].set_yticks(y_pos)
axes_pc[1].set_yticklabels([filenames[i] if i < len(filenames) else f'C{i}' for i in range(num_classes)], fontsize=8)
axes_pc[1].set_xlabel('Recall', fontsize=12, fontweight='bold')
axes_pc[1].set_title('Recall por Clase', fontsize=14, fontweight='bold')
axes_pc[1].set_yticklabels(unique_class_names_exp2, fontsize=8)
axes_pc[1].grid(True, alpha=0.3, axis='x')

axes_pc[2].barh(y_pos, f1_per_class_array_exp2, color='lightgreen', edgecolor='darkgreen', alpha=0.7)
axes_pc[2].set_yticks(y_pos)
axes_pc[2].set_yticklabels([filenames[i] if i < len(filenames) else f'C{i}' for i in range(num_classes)], fontsize=8)
axes_pc[2].set_xlabel('F1-Score', fontsize=12, fontweight='bold')
axes_pc[2].set_title('F1-Score por Clase', fontsize=14, fontweight='bold')
axes_pc[2].set_yticklabels(unique_class_names_exp2, fontsize=8)
axes_pc[2].grid(True, alpha=0.3, axis='x')

plt.suptitle('Análisis por Clase - LABEL-SMOOTH', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(output_dir_exp2 / 'per_class_analysis.png', dpi=300, bbox_inches='tight')
print(f"✓ Guardado: per_class_analysis.png")
plt.close()

# 9. RESUMEN.txt
f1_per_class_list_exp2 = [(i, f1_value) for i, f1_value in enumerate(f1_per_class_array_exp2)]
class_f1_sorted_exp2 = sorted(f1_per_class_list_exp2, key=lambda x: x[1], reverse=True)

summary_exp2 = f"""
{'='*80}
RESUMEN EJECUTIVO - G4-RESULTS-LABEL-SMOOTH
{'='*80}

📊 PERFORMANCE:
  • Test Accuracy:       {test_acc_exp2:.4f}
  • Macro F1-Score:      {macro_f1_exp2:.4f}
  • Macro Precision:     {macro_precision_exp2:.4f}
  • Macro Recall:        {macro_recall_exp2:.4f}
  • Top-3 Accuracy:      {top3_acc_exp2:.4f}

⚙️ CONFIGURACIÓN:
  • Dropout:             0.3
  • Class Weights:       Desactivado
  • Label Smoothing:     0.1
  • Best Epoch:          {best_epoch_exp2}

📈 TOP 5 CLASES (F1-Score):
"""
for rank, (class_idx, f1_value) in enumerate(class_f1_sorted_exp2[:5], 1):
    class_name = filenames[class_idx] if class_idx < len(filenames) else f'Class {class_idx}'
    summary_exp2 += f"  {rank}. {class_name:20s} | F1: {f1_value:.4f}\n"

    class_name = unique_class_names_exp2[class_idx] if class_idx < len(unique_class_names_exp2) else f'Class {class_idx}'

with open(output_dir_exp2 / 'RESUMEN.txt', 'w', encoding='utf-8') as f:
    f.write(summary_exp2)
print(f"✓ Guardado: RESUMEN.txt")

# Resultados en diccionario
exp2_results = {
    'experiment': 'G4-RESULTS-LABEL-SMOOTH',
    'dropout': 0.3,
    'class_weights': False,
    'label_smoothing': 0.1,
    'test_accuracy': test_acc_exp2,
    'test_macro_f1': macro_f1_exp2,
    'test_top3_accuracy': top3_acc_exp2,
    'best_epoch': best_epoch_exp2
}

print(f"{'='*80}")
print("✅ EXPERIMENTO 2 (LABEL-SMOOTH) COMPLETADO")
print(f"\n{'='*80}")



Iniciando Experimento 2: G4-RESULTS-LABEL-SMOOTH
📁 Directorio: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\G4-RESULTS-LABEL-SMOOTH

Entrenando con Label Smoothing 0.1 y Dropout 0.3...


Epoch   1/50 | Train Loss: 3.3199 | Val Acc: 0.0791


Epoch   5/50 | Train Loss: 3.2570 | Val Acc: 0.1007


Epoch  10/50 | Train Loss: 3.0678 | Val Acc: 0.1511


Epoch  15/50 | Train Loss: 2.9985 | Val Acc: 0.1223


Epoch  20/50 | Train Loss: 2.8664 | Val Acc: 0.2158


Epoch  25/50 | Train Loss: 2.7748 | Val Acc: 0.2086


Epoch  30/50 | Train Loss: 2.6158 | Val Acc: 0.2086


Epoch  35/50 | Train Loss: 2.5361 | Val Acc: 0.2446
Early stopping at epoch 35



✓ LABEL-SMOOTH completado:
  Test Accuracy: 0.2601
  Macro F1: 0.0781
  Top-3 Acc: 0.4566
✓ Guardado: training_log.csv
✓ Guardado: metrics.csv
✓ Guardado: per_class_metrics.csv
✓ Guardado: confusion_matrix.csv
✓ Guardado: config.json
✓ Guardado: training_curves.png
✓ Guardado: confusion_matrix.png
✓ Guardado: per_class_analysis.png
✓ Guardado: RESUMEN.txt
✅ EXPERIMENTO 2 (LABEL-SMOOTH) COMPLETADO



## 📊 Comparación de Experimentos UMAP

In [28]:
# COMPARACIÓN DE LOS 3 EXPERIMENTOS
print("\n" + "="*80)
print("COMPARACIÓN DE LOS 3 EXPERIMENTOS")
print("="*80)

# DataFrame comparativo
all_results = [exp0_results, exp1_results, exp2_results]
df_comparison = pd.DataFrame(all_results)

print("\n📊 Tabla Comparativa:")
print(df_comparison[['experiment', 'test_accuracy', 'test_macro_f1', 'test_top3_accuracy']].to_string(index=False))

# Análisis de mejoras
best_f1_idx = df_comparison['test_macro_f1'].idxmax()
best_f1_exp = df_comparison.loc[best_f1_idx, 'experiment']

improvement_exp1 = (exp1_results['test_macro_f1'] - exp0_results['test_macro_f1']) * 100
improvement_exp2 = (exp2_results['test_macro_f1'] - exp0_results['test_macro_f1']) * 100

print(f"\n📈 Análisis:")
print(f"  Mejor Macro-F1: {best_f1_exp}")
print(f"  Mejora Exp1 vs Baseline: {improvement_exp1:+.2f}%")
print(f"  Mejora Exp2 vs Baseline: {improvement_exp2:+.2f}%")

# Guardar comparación CSV
comparison_csv_path = ROOT_PATH / 'experiments_comparison.csv'
df_comparison.to_csv(comparison_csv_path, index=False)
print(f"\n✓ Guardado: {comparison_csv_path}")

print(f"\n{'='*80}")
print("✅ COMPARACIÓN COMPLETADA")
print(f"{'='*80}")


COMPARACIÓN DE LOS 3 EXPERIMENTOS

📊 Tabla Comparativa:
              experiment  test_accuracy  test_macro_f1  test_top3_accuracy
              G4-RESULTS       0.265896       0.092025            0.531792
G4-RESULTS-CLASS-WEIGHTS       0.150289       0.025359            0.208092
 G4-RESULTS-LABEL-SMOOTH       0.260116       0.078113            0.456647

📈 Análisis:
  Mejor Macro-F1: G4-RESULTS
  Mejora Exp1 vs Baseline: -6.67%
  Mejora Exp2 vs Baseline: -1.39%

✓ Guardado: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP\experiments_comparison.csv

✅ COMPARACIÓN COMPLETADA


## 🔍 Verificación Final de Archivos Generados

In [29]:
# VERIFICACIÓN DE ARCHIVOS GENERADOS
print("\n" + "="*80)
print("🔍 VERIFICACIÓN DE ARCHIVOS GENERADOS")
print("="*80)

print(f"\n📂 ROOT_PATH: {ROOT_PATH}")
print(f"📝 Tipo de dataset: UMAP Embeddings")

# Archivos requeridos por experimento
REQUIRED_FILES_PER_EXPERIMENT = [
    'best_model.pt',
    'config.json',
    'training_log.csv',
    'metrics.csv',
    'per_class_metrics.csv',
    'confusion_matrix.csv',
    'confusion_matrix.png',
    'training_curves.png',
    'per_class_analysis.png',
    'RESUMEN.txt'
]

# Archivos de comparación en ROOT_PATH
REQUIRED_FILES_BASE = [
    'experiments_comparison.csv',
    'experiments_comparison.png'
]

# Carpetas de experimentos
experiment_folders = [
    'G4-RESULTS',
    'G4-RESULTS-CLASS-WEIGHTS',
    'G4-RESULTS-LABEL-SMOOTH'
]

# Verificar cada experimento
all_valid = True
missing_files = []

for folder_name in experiment_folders:
    folder_path = ROOT_PATH / folder_name
    print(f"\n📂 {folder_name}:")
    
    if not folder_path.exists():
        print(f"  ❌ Carpeta no existe")
        all_valid = False
        continue
    
    for required_file in REQUIRED_FILES_PER_EXPERIMENT:
        file_path = folder_path / required_file
        if file_path.exists():
            file_size = file_path.stat().st_size
            print(f"  ✅ {required_file:30s} ({file_size:,} bytes)")
        else:
            print(f"  ❌ {required_file:30s} FALTA")
            missing_files.append(f"{folder_name}/{required_file}")
            all_valid = False

# Verificar archivos de comparación
print(f"\n📂 Archivos de comparación en ROOT_PATH:")
for required_file in REQUIRED_FILES_BASE:
    file_path = ROOT_PATH / required_file
    if file_path.exists():
        file_size = file_path.stat().st_size
        print(f"  ✅ {required_file:30s} ({file_size:,} bytes)")
    else:
        print(f"  ❌ {required_file:30s} FALTA")
        missing_files.append(f"ROOT/{required_file}")
        all_valid = False

# Resumen final
print(f"\n{'='*80}")
if all_valid:
    print("✅ VERIFICACIÓN EXITOSA - Todos los archivos se han generado correctamente")
else:
    print("⚠️ VERIFICACIÓN INCOMPLETA - Faltan algunos archivos:")
    for missing in missing_files:
        print(f"  - {missing}")
print(f"{'='*80}")

# Estadísticas
total_required = len(experiment_folders) * len(REQUIRED_FILES_PER_EXPERIMENT) + len(REQUIRED_FILES_BASE)
print(f"\n📊 Resumen:")
print(f"  • Experimentos: {len(experiment_folders)}")
print(f"  • Archivos por experimento: {len(REQUIRED_FILES_PER_EXPERIMENT)}")
print(f"  • Archivos de comparación: {len(REQUIRED_FILES_BASE)}")
print(f"  • Total archivos requeridos: {total_required}")
print(f"  • ROOT_PATH: {ROOT_PATH}")


🔍 VERIFICACIÓN DE ARCHIVOS GENERADOS

📂 ROOT_PATH: C:\Users\Los milluelitos repo\Desktop\experimento tesis\transformer-asl-classification\G4-EMBEDDING FRAME A FRAME UMAP
📝 Tipo de dataset: UMAP Embeddings

📂 G4-RESULTS:
  ❌ best_model.pt                  FALTA
  ✅ config.json                    (507 bytes)
  ✅ training_log.csv               (4,601 bytes)
  ✅ metrics.csv                    (209 bytes)
  ✅ per_class_metrics.csv          (1,691 bytes)
  ✅ confusion_matrix.csv           (1,833 bytes)
  ✅ confusion_matrix.png           (675,717 bytes)
  ✅ training_curves.png            (596,963 bytes)
  ✅ per_class_analysis.png         (400,340 bytes)
  ✅ RESUMEN.txt                    (1,641 bytes)

📂 G4-RESULTS-CLASS-WEIGHTS:
  ✅ best_model.pt                  (19,541,557 bytes)
  ✅ config.json                    (511 bytes)
  ✅ training_log.csv               (1,622 bytes)
  ✅ metrics.csv                    (212 bytes)
  ✅ per_class_metrics.csv          (1,411 bytes)
  ✅ confusion_matrix